In [1]:
import pandas as pd

In [2]:
tesis = pd.read_csv("../Datos/tesis_final.csv")
tesis.head()

,idPaciente,Mes,TAS,Adherencia,Peso,Altura,IMC,DBT,Sexo,Edad,Fuma,ant_HTA,tas_basal,ICC
0,4026,1,119.000000,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459
1,4026,2,127.000000,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459
2,4026,3,140.000000,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459
3,4026,4,146.712710,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459
4,4026,5,177.708084,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459


In [3]:
tesis = tesis.drop(["Peso", "Altura", "IMC", "DBT", "Fuma", "ant_HTA", "ICC"], axis=1)

In [4]:
# Crear Adherencia_Acum: variable de indice de performance de adherencia al tratamiento hasta el momento t
tesis["Adherencia_Acumulada"] = tesis.groupby("idPaciente")["Adherencia"].expanding().mean().to_list()
# Crear covariable Adherencia_Total no dependiente del tiempo con el performance final de cada paciente
tesis["Adherencia_Total"] = tesis["idPaciente"].map(tesis.groupby("idPaciente")["Adherencia"].mean().to_dict())

tesis["Adherencia_Perfecta"] = (tesis["Adherencia_Total"] == 1).astype(int)

In [5]:
# Crear TAS_Media_Acum: variable de TAS media hasta el momento t
tesis["TAS_Media_Acumulada"] = tesis.groupby("idPaciente")["TAS"].expanding().mean().to_list()

In [7]:
tesis["Intercept"] = 1

In [8]:
tesis["Adherencia_lag1"] = tesis.groupby("idPaciente")["Adherencia"].shift(1).fillna(0)

tesis["TAS_lag1"] = tesis.groupby("idPaciente")["TAS"].shift(1).fillna(tesis["tas_basal"])

tesis["Adherencia_Acumulada_lag2"] = tesis.groupby("idPaciente")["Adherencia_Acumulada"].shift(2).fillna(0)

tesis["TAS_Media_Acumulada_lag2"] = tesis.groupby("idPaciente")["TAS_Media_Acumulada"].shift(2).fillna(tesis["tas_basal"])

In [9]:
tesis["Mes*Adherencia"] = tesis["Mes"] * tesis["Adherencia"]
tesis["Mes*Adherencia_Perfecta"] = tesis["Mes"] * tesis["Adherencia_Perfecta"]
tesis["Mes*Adherencia_Total"] = tesis["Mes"] * tesis["Adherencia_Total"]
tesis["Mes*Adherencia_lag1"] = tesis["Mes"] * tesis["Adherencia_lag1"]
tesis["Mes*Adherencia_Acumulada"] = tesis["Mes"] * tesis["Adherencia_Acumulada"]

In [10]:
tesis.to_csv("../Datos/tesis_final_preprocesado.csv", index=False)